In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping
from random import sample
import seaborn as sns

%matplotlib inline

Using TensorFlow backend.


In [2]:
dftrain=pd.read_csv('Train.csv')
dftest=pd.read_csv('Test.csv')

In [3]:
dftrain.index=dftrain['Datetime']
dftrain=dftrain.drop('Datetime',axis=1)

In [4]:
dftest.index=dftest['Datetime']
dftest=dftest.drop('Datetime',axis=1)

In [5]:
dataset  =dftrain.values #numpy.ndarray
dataset = dataset.astype('float32')
print(dataset.shape)
dataset=np.array(dataset)
dataset = np.reshape(dataset, (-1, 1))


(18288, 2)


In [6]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
   X, Y = [], []
   for i in range(len(dataset)-look_back-1):
       a = dataset[i:(i+look_back), 0]
       X.append(a)
       Y.append(dataset[i + look_back, 0])
   return np.array(X), np.array(Y)


In [7]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [8]:
scaler = MinMaxScaler(feature_range=(0,1))
dataset = scaler.fit_transform(dataset)
train_size = int(len(dataset)*0.80)
test_size = len(dataset) - train_size
#train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

# reshape into X=t and Y=t+1
look_back = 30
xtrain,ytrain = create_dataset(train, look_back)
xtest,ytest = create_dataset(test, look_back)


In [9]:
#xtrain=np.reshape(xtrain,(xtrain.shape[0],1,xtrain.shape[1]))
#xtest=np.reshape(xtest,(xtest.shape[0],1,xtest.shape[1]))
xtrain.shape,ytrain.shape,xtest.shape,ytest.shape

((29229, 30), (29229,), (7285, 30), (7285,))

In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape=(xtrain.shape[0], xtrain.shape[1])))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])

history = model.fit(xtrain, ytrain, epochs=20, batch_size=70,
                    callbacks=[EarlyStopping(monitor='loss', 
                    patience=10)], verbose=1, shuffle=False)

#Training Phase
model.summary()

In [ ]:
train_predict = model.predict(xtrain)
test_predict = model.predict(xtest)

In [ ]:
test_predict.shape,ytest.shape,train_predict.shape,ytrain.shape

In [ ]:
# invert predictions
trainPredict = scaler.inverse_transform(train_predict)
trainY = scaler.inverse_transform([ytrain])
testPredict = scaler.inverse_transform(test_predict).reshape(-1,1)
testY = scaler.inverse_transform([ytest])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# invert predictions

#test_predict_inverse = scaler.inverse_transform([ytest])

#test_predict=np.array(test_predict).reshape(test_predict,(test_predict.shape[0]))

print('Test Mean Absolute Error:',mean_absolute_error(ytest, test_predict))
print('Test Root Mean Squared Error:', np.sqrt(mean_squared_error(ytest, 
                                               test_predict)))

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label="Train Loss")
plt.plot(history.history['loss'], label="Test Loss")
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(loc='upper right')
plt.show();

In [ ]:
aa=[x for x in range(200)]
plt.figure(figsize=(8,4))
plt.plot(aa, ytest[0:200], marker='.', label="actual")
plt.plot(aa, test_predict[0:200], 'r', label="prediction")
# plt.tick_params(left=False, labelleft=True) #remove ticks
plt.tight_layout()
sns.despine(top=True)
plt.subplots_adjust(left=0.07)
plt.ylabel('Global_active_power', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();


In [ ]:
ytest.shape[0],train_size,test_predict.shape[0],test_size

In [ ]:
aa=[x for x in range(200)]
plt.figure(figsize = (18,9))
plt.plot(range(0,ytest.shape[0]),ytest[0:test_size],color='b',label='True')
plt.plot(range(0,test_predict.shape[0]), test_predict,color='orange', label='Prediction')
#plt.xticks(range(0,df.shape[0],50),df['Date'].loc[::50],rotation=45)
plt.xlabel('Date')
plt.ylabel('Mid Price')
plt.legend(fontsize=18)
plt.show()